In [1]:
%matplotlib Tk
import numpy as np
from sympy import Matrix
import matplotlib.pyplot as plt
from IPython.display import clear_output
from matplotlib.animation import FuncAnimation, ArtistAnimation
import matplotlib

plt.style.use('seaborn-pastel')

In [2]:
def excite_atoms(positions, states):
    for position in positions:
        states[position[0], position[1], position[2]] = 1
    return

def random_seeds_displacement(N_atoms_per_row, N_seeds):
    seeds_positions = np.random.choice(range(N_atoms_per_row**3), N_seeds, replace=False)
    z_positions = [seeds_positions[i]//(N_atoms_per_row**2) for i in range(N_seeds)]
    x_positions = [(seeds_positions[i]%(N_atoms_per_row**2))//N_atoms_per_row for i in range(N_seeds)]
    y_positions = [(seeds_positions[i]%(N_atoms_per_row**2))%N_atoms_per_row for i in range(N_seeds)]
    return x_positions, y_positions, z_positions
        
def get_neighbourhood(excited_atom, N_atoms_per_row):
    neighbours = [
                (excited_atom[0]-1, excited_atom[1], excited_atom[2]), 
                (excited_atom[0]+1, excited_atom[1], excited_atom[2]),
                (excited_atom[0], excited_atom[1]-1, excited_atom[2]), 
                (excited_atom[0], excited_atom[1]+1, excited_atom[2]),
                (excited_atom[0], excited_atom[1], excited_atom[2]-1), 
                (excited_atom[0], excited_atom[1], excited_atom[2]+1)
                ]
    for neighbour in neighbours.copy():
        if -1 in list(neighbour) or N_atoms_per_row in list(neighbour):
            neighbours.remove(neighbour)
    return neighbours

def unzip(list_of_touples):
    '''
    works only for 3D case.
    '''
    if len(list_of_touples) == 0:
        return [], [], []
    if list_of_touples is not list:
        list_of_touples = list(list_of_touples)
    aus_pos = np.zeros([len(list_of_touples[0]), len(list_of_touples)], dtype=int)
    for i in range(len(list_of_touples[0])):
        for j in range(len(list_of_touples)):
            aus_pos[i,j] = list_of_touples[j][i]
    return list(aus_pos[0,:]), list(aus_pos[1,:]), list(aus_pos[2,:])

In [3]:
from turtle import color


N_steps = 100
N_atoms_per_row = 10
mu_exct_on_resonance = 0.2
p_exct_OFF = 0.1
p_spontaneus_decay = 0.1

N_seeds = np.random.poisson(mu_exct_on_resonance) # extract from a poassonian the inital seeds
x_excited, y_excited, z_excited = random_seeds_displacement(N_atoms_per_row=N_atoms_per_row, N_seeds=N_seeds) # extract the random positions of the seeds
excited_atoms = list(zip(x_excited, y_excited, z_excited))

fig = plt.figure(figsize=(13,10))
ims = []

for i in range(N_steps):
    # first phase: check if the neighbours are already excited excite them with a certain probability.
    for excited_atom in excited_atoms.copy():
        neighbours = get_neighbourhood(excited_atom, N_atoms_per_row)
        for neighbour in neighbours:
            if neighbour not in excited_atoms and np.random.random()<p_exct_OFF:
                excited_atoms.append(neighbour)
    # second phase: spontaneous decay of atoms.
    for excited_atom in excited_atoms.copy():
        if np.random.random()<p_spontaneus_decay:
            excited_atoms.remove(excited_atom)

    N_seeds = np.random.poisson(mu_exct_on_resonance) # extract from a poassonian the inital seeds
    x_excited, y_excited, z_excited = random_seeds_displacement(N_atoms_per_row=N_atoms_per_row, N_seeds=N_seeds) # extract the random positions of the seeds
    excited_atoms = excited_atoms + list(zip(x_excited, y_excited, z_excited))

    states = np.zeros([N_atoms_per_row, N_atoms_per_row, N_atoms_per_row])
    excite_atoms(excited_atoms, states)
    final_pos = list(unzip(excited_atoms))
    x_pos, y_pos, z_pos = final_pos[0]+np.random.random(len(final_pos[0]))*0.4-0.2, final_pos[1]+np.random.random(len(final_pos[0]))*0.4-0.2, final_pos[2]+np.random.random(len(final_pos[0]))*0.4-0.2

    im = plt.axes(projection ="3d")
    im.scatter3D(x_pos, y_pos, z_pos, marker="o", alpha=1, s=70, color="blue")
    im.plot(x_pos, y_pos, marker="o", alpha=0.3, color="gray", linestyle="")

    im.set_zlim(0,N_atoms_per_row)
    im.set_xlim(0,N_atoms_per_row)
    im.set_ylim(0,N_atoms_per_row)

    im.view_init(30, 80/N_steps*i)

    ims.append([im])

ani = ArtistAnimation(fig, ims, interval=100, blit=True, repeat_delay=10)
#ani.save('active_phase_3d.mp4')

: 

In [8]:
N_atoms_per_row = 50
print("N atoms: ", N_atoms_per_row*N_atoms_per_row*N_atoms_per_row)
mu_exct_on_resonance = 6
p_exct_OFF = 0.1
p_spontaneus_decay = 0.1
N_steps = 80

states = np.zeros([N_atoms_per_row, N_atoms_per_row, N_atoms_per_row])
excited_atoms = simulation(mu_exct_on_resonance, p_exct_OFF, p_spontaneus_decay, N_atoms_per_row, N_steps)
excite_atoms(excited_atoms, states)
final_pos = list(unzip(excited_atoms))
#print(final_pos)
x_pos, y_pos, z_pos = final_pos[0], final_pos[1], final_pos[2]
plt.figure(figsize=(13,10))
ax = plt.axes(projection ="3d")
ax.scatter3D(x_pos, y_pos, z_pos, marker="o", alpha=1, s=100)

ax.set_zlim(0,N_atoms_per_row)
ax.set_xlim(0,N_atoms_per_row)
ax.set_ylim(0,N_atoms_per_row)
plt.show()

N atoms:  125000


NameError: name 'simulation' is not defined